# Aprendizagem por regras

In [ ]:
!pip install Orange3

In [1]:
import Orange

## Base risco de crédito

In [2]:
# importando pelo orange, pode ser em csv
# a base foi modificada para colocar um c# na variável que queremos prever
base_risco_credito = Orange.data.Table('risco_credito_regras.csv')

In [3]:
base_risco_credito

[[ruim, alta, nenhuma, 0_15 | alto],
 [desconhecida, alta, nenhuma, 15_35 | alto],
 [desconhecida, baixa, nenhuma, 15_35 | moderado],
 [desconhecida, baixa, nenhuma, acima_35 | alto],
 [desconhecida, baixa, nenhuma, acima_35 | baixo],
 ...
]

In [4]:
base_risco_credito.domain

[historia, divida, garantias, renda | risco]

In [5]:
#treinando o algoritmo
cn2 = Orange.classification.rules.CN2Learner()
regras_risco_credito = cn2(base_risco_credito)

In [6]:
#Mostrando as regras
for regras in regras_risco_credito.rule_list:
  print(regras)

IF renda==0_15 THEN risco=alto 
IF historia==boa AND divida!=alta THEN risco=baixo 
IF historia==boa AND garantias!=nenhuma THEN risco=baixo 
IF historia==boa AND renda!=15_35 THEN risco=baixo 
IF historia==boa THEN risco=moderado 
IF divida==alta THEN risco=alto 
IF historia!=desconhecida THEN risco=moderado 
IF garantias==adequada THEN risco=baixo 
IF renda==15_35 THEN risco=moderado 
IF historia==desconhecida THEN risco=baixo 
IF TRUE THEN risco=alto 


In [7]:
# Previsões
# história boa, dívida alta, garantias nenhuma, renda > 35
# história ruim, dívida alta, garantias adequada, renda < 15
previsoes = regras_risco_credito([['boa', 'alta', 'nenhuma', 'acima_35'], ['ruim', 'alta', 'adequada', '0_15']])
previsoes

array([1, 0])

In [8]:
#Colocando label nas previsões
base_risco_credito.domain.class_var.values

('alto', 'baixo', 'moderado')

In [9]:
for i in previsoes:
  #print(i)
  print(base_risco_credito.domain.class_var.values[i])

baixo
alto


## Base credit data

In [11]:
base_credit = Orange.data.Table('/content/credit_data_regras.csv')
base_credit.domain

[income, age, loan | default]

In [13]:
# Dividindo a base
base_dividida = Orange.evaluation.testing.sample(base_credit, n = 0.25)
base_dividida

([[23007.4, 48.9725, 2296.8 | 0],
  [20063.1, 24.2783, 2495.13 | 1],
  [55613.5, 24.3771, 4255.25 | 0],
  [62114.9, 30.7495, 12115.9 | 1],
  [58507.6, 48.9774, 694.135 | 0],
  ...
 ],
 [[49264.3, 19.0065, 801.78 | 0],
  [40053.7, 27.9385, 44.5272 | 0],
  [24114, 52.3358, 3900.83 | 0],
  [63391.6, 34.7393, 190.889 | 0],
  [61878.3, 33.0064, 567.669 | 0],
  ...
 ])

In [14]:
base_treinamento = base_dividida[1]
base_teste = base_dividida[0]

In [15]:
len(base_treinamento), len(base_teste)

(1500, 500)

In [16]:
cn2 = Orange.classification.rules.CN2Learner()
regras_credit = cn2(base_treinamento)

In [17]:
for regras in regras_credit.rule_list:
  print(regras)

IF age>=34.9257164876908 THEN default=0 
IF age>=34.795262857340305 THEN default=1 
IF loan<=2507.64970973955 AND income>=20568.891305010802 THEN default=0 
IF loan>=7708.31562497011 AND loan>=9601.375482171099 THEN default=1 
IF loan>=7708.31562497011 AND loan>=9595.28628892989 THEN default=0 
IF loan>=7708.31562497011 AND age>=20.9909665295854 THEN default=1 
IF income>=57596.3540980592 AND age>=20.3008601283655 THEN default=0 
IF age<=18.075335860718 AND age>=18.075335860718 THEN default=1 
IF age<=18.1760434475727 AND age>=18.1760434475727 THEN default=1 
IF age<=18.3097456344403 AND age>=18.3097456344403 THEN default=1 
IF age<=18.4162362311035 AND income>=32197.6207010448 THEN default=0 
IF age>=33.9493660898879 AND income>=44405.28066363191 THEN default=0 
IF income<=37162.8882177174 AND loan>=3928.3039085182704 THEN default=1 
IF age>=34.645415822044 THEN default=0 
IF loan>=6043.14310633161 AND age>=23.6371360039338 THEN default=1 
IF loan>=6994.48780081424 AND age>=18.6213070

In [18]:
# Fazendo previsoes
previsoes = Orange.evaluation.testing.TestOnTestData(base_treinamento, base_teste, [lambda testdata: regras_credit])

In [19]:
Orange.evaluation.CA(previsoes)

array([0.966])

#Classificador base - Majority learner

Útil para verificar se o algoritmo utilizado está tendo um bom desempenho. No caso ele só calcula qual a classe mais provável.
Serve para comparar se o algoritmo utilizado é melhor (ou pior) do que eu só chutar na classe que aparece mais vezes no banco de dados


##Base census

In [25]:
base_credit = Orange.data.Table('credit_data_regras.csv')
base_credit.domain

[income, age, loan | default]

In [26]:
majority = Orange.classification.MajorityLearner()
previsoes = Orange.evaluation.testing.TestOnTestData(base_credit, base_credit, [majority])

In [27]:
Orange.evaluation.CA(previsoes)

array([0.8585])

In [28]:
for registro in base_credit:
  print(registro.get_class())

0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
1
1
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
1
0
1
1
0
0
0
1
1
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
1
0
0
0
1
0
1
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
1
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
1
0
0
0
0
0
1
0
1
0
0
0
1
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
1
1
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
1
0
0
0
1
0
0
1
0
0
0
0


In [29]:
from collections import Counter
Counter(str(registro.get_class()) for registro in base_credit)

Counter({'0': 1717, '1': 283})

In [30]:
1717 / 2000

0.8585

Interpretação: qualquer algoritmo que não acertar 85% das vezes, é pior do que chutar na maioria.